In [29]:
import numpy as np
# from qhdopt import QHD
import sys
sys.path.insert(1, "../../../")
from config import *
from os import getenv
from os.path import join
DWAVE_API_KEY = getenv("DWAVE_API_KEY")
import json
from dwave.system import DWaveSampler, EmbeddingComposite, FixedEmbeddingComposite

In [30]:
def list_to_vec(sample_list, P):
    # sample_list = a sample from the response of the D-Wave sampler
    # P = the full precision matrix for d-dim vector
    list_len = len(sample_list)
    binary_vec = np.empty((list_len))
    for k in np.arange(list_len):
        binary_vec[k] = sample_list['%s'%k]
        
    return P @ binary_vec

def get_qubo(Q, b, Q_c, b_c, H, sigma):
    # We assume all continuous variables are between 0 and 1.
    
    Q_qhd = H.T @ (Q / 2 + sigma * Q_c.T @ Q_c) @ H
    R_qhd = (b - 2 * sigma * b_c.T @ Q_c) @ H

    # Build QUBO dict
    QM = dict()
    for i in range(len(H[0])):
        if (np.abs(Q_qhd[i,i] + R_qhd[i]) > 0):
            QM[str(i),str(i)] = Q_qhd[i,i] + R_qhd[i]
        for j in np.arange(i+1, len(H[0])):
            if (np.abs(Q_qhd[i,j]) > 0):
                QM[str(i),str(j)] = 2 * Q_qhd[i,j]

    # maximum strength
    max_interaction_qhd = np.max(np.abs(np.array(list(QM.values()))))
    
    return QM, max_interaction_qhd

def get_delay_list(Q_c, dimension, resolution, embedding):
    var_equality_constraint = []
    for k in range(len(Q_c)):
        for j in range(dimension):
            if Q_c[k][j] != 0:
                var_equality_constraint.append(j)
    var_equality_constraint = np.unique(var_equality_constraint)

    qubit_list = []
    for var in var_equality_constraint:
        var_to_qbt = np.arange(var*resolution, (var+1)*resolution)
        qubit_list.append(var_to_qbt)
    qubit_list = np.unique(qubit_list)

    delay_list = []
    for qubit in qubit_list:
        physical_qbt = embedding[str(qubit)]
        for j in physical_qbt:
            delay_list.append(j)
    return delay_list

In [31]:
device_name = "Advantage_system6.4"
# device_name = "Advantage2_prototype2.6"

if device_name == "Advantage_system6.4":
    device_label = "adv"
    USE_EXISTING_EMBEDDING = True
elif device_name == "Advantage2_prototype2.6":
    device_label = "adv2"
    USE_EXISTING_EMBEDDING = False

qaa_schedule = [[0, 0], [20, 1]]
qhd_schedule = [[0,0],[400, 0.3],[640, 0.6],[800,1]]

In [32]:

# Benchmark information
dimension = 75
sparsity = 5
num_instances = 50# TODO: modify
num_shots = 1000
resolution = 8

if dimension == 5:
    benchmark_name = f"QP-5d"
else:
    benchmark_name = f"QP-{dimension}d-{sparsity}s"

benchmark_dir = join(DATA_DIR_QP, benchmark_name)
print(f"Current benchmark: {benchmark_name}")

Current benchmark: QP-75d-5s


Running QAA

In [ ]:
# Quantum Adiabatic Algorithm (QAA)
# Use the Radix-2 encoding of real numbers

print('Device:', device_name)

# QUBO parameters
sigma = 0 # penalty coefficient, set as 100 if equality_constraint != 0
p = np.array([2**(-(int(np.log2(resolution))-i+1)) for i in range(int(np.log2(resolution))+1)])
p[0] = 2**(-(int(np.log2(resolution))))
Id = np.identity(dimension)
P = np.kron(Id, p) # Precision vector for Radix-2 encoding

# QPU parameters
embedding_filename = join(benchmark_dir, f"config/advantage6_qaa_rez{resolution}_embedding.json")
if USE_EXISTING_EMBEDDING:
    # Load Embedding from json
    with open(embedding_filename, 'r', encoding='utf-8') as f:
        embedding = json.loads(f.read())
    
    #js_content = open(embedding_filename)
    #embedding = json.loads(js_content)


for instance in range(num_instances):
    print(f"Run standard QAA (Radix-2) on instance No. {instance} from {benchmark_name}.")
    
    # Load instance data
    instance_filename = join(benchmark_dir, f"instance_{instance}/instance_{instance}.npy")
    with open(instance_filename, 'rb') as f:
        Q = np.load(f)
        b = np.load(f)
        Q_c = np.load(f)
        b_c = np.load(f)
    
    # Build QUBO dict
    qaa_samples = np.zeros(shape=(num_shots, dimension))
    QM, max_interaction = get_qubo(Q, b, Q_c, b_c, P, sigma)
    chain_strength = 1.1 * max_interaction
    
    if (USE_EXISTING_EMBEDDING == False) and (instance == 0):
        # Generate a new embedding and save it to {benchmark_name}/config
        qpu = DWaveSampler(token=DWAVE_API_KEY, solver=device_name)
        sampler = EmbeddingComposite(qpu)
        response_qaa = sampler.sample_qubo(QM, 
                                    chain_strength=chain_strength, 
                                    num_reads=num_shots,
                                    anneal_schedule=qaa_schedule,
                                    return_embedding=True)
        embedding = response_qaa.info["embedding_context"]["embedding"]
        with open(embedding_filename, "w") as outfile:
            json.dump(embedding, outfile)
        print(f"Minor embedding saved.")
    else: 
        qpu = DWaveSampler(token=DWAVE_API_KEY, solver=device_name)
        sampler = FixedEmbeddingComposite(qpu, embedding)
        response_qaa = sampler.sample_qubo(QM, 
                                    chain_strength=chain_strength, 
                                    num_reads=num_shots,
                                    anneal_schedule=qaa_schedule)
    
    samples_list_qaa = list(response_qaa.samples())
    for i in range(num_shots):
        qaa_samples[i] = list_to_vec(samples_list_qaa[i], P)
    qaa_avg_runtime = response_qaa.info['timing']['qpu_access_time'] * 1e-6 / num_shots
    np.save(join(benchmark_dir, f"instance_{instance}/dwave_{device_label}_qaa_rez{resolution}_runtime_{instance}.npy"), qaa_avg_runtime)
    # Save QAA samples
    qaa_filename = join(benchmark_dir, f"instance_{instance}/dwave_{device_label}_qaa_rez{resolution}_sample_{instance}.npy")
    with open(qaa_filename, 'wb') as f:
        np.save(f, qaa_samples)
    print(f"D-Wave standard QAA: data saved.")
    
    # Finish processing instance 
    print(f"Instance No. {instance} from {benchmark_name}: completed.\n")

Running QHD

In [ ]:
print('Device:', device_name)

# QUBO parameters
sigma = 0 # penalty coefficient, set as 100 if equality_constraint != 0
h = np.ones(resolution) / resolution
Id = np.identity(dimension)
H = np.kron(Id, h) # Precision vector for Radix-2 encoding

# QPU parameters
embedding_filename = join(benchmark_dir, f"config/advantage6_qhd_rez{resolution}_embedding.json")
if USE_EXISTING_EMBEDDING:
    # Load Embedding from json
    with open(embedding_filename, 'r', encoding='utf-8') as f:
        embedding = json.loads(f.read())
    
    #js_content = open(embedding_filename)
    #embedding = json.loads(js_content)


for instance in range(num_instances):
    print(f"Run QHD on instance No. {instance} from {benchmark_name}.")
    
    # Load instance data
    instance_filename = join(benchmark_dir, f"instance_{instance}/instance_{instance}.npy")
    with open(instance_filename, 'rb') as f:
        Q = np.load(f)
        b = np.load(f)
        Q_c = np.load(f)
        b_c = np.load(f)
    
    # Build QUBO dict
    qhd_samples = np.zeros(shape=(num_shots, dimension))
    QM, max_interaction = get_qubo(Q, b, Q_c, b_c, H, sigma)
    chain_strength = 1.1 * max_interaction
    
    if (USE_EXISTING_EMBEDDING == False) and (instance == 0):
        # Generate a new embedding and save it to {benchmark_name}/config
        qpu = DWaveSampler(token=DWAVE_API_KEY, solver=device_name)
        sampler = EmbeddingComposite(qpu)
        response_qhd = sampler.sample_qubo(QM, 
                                    chain_strength=chain_strength, 
                                    num_reads=num_shots,
                                    anneal_schedule=qhd_schedule,
                                    return_embedding=True)
        embedding = response_qhd.info["embedding_context"]["embedding"]
        with open(embedding_filename, "w") as outfile:
            json.dump(embedding, outfile)
        print(f"Minor embedding saved.")
    else: 
        qpu = DWaveSampler(token=DWAVE_API_KEY, solver=device_name)
        sampler = FixedEmbeddingComposite(qpu, embedding)
        response_qhd = sampler.sample_qubo(QM, 
                                    chain_strength=chain_strength, 
                                    num_reads=num_shots,
                                    anneal_schedule=qaa_schedule)
    
    samples_list_qhd = list(response_qhd.samples())
    for i in range(num_shots):
        qhd_samples[i] = list_to_vec(samples_list_qhd[i], H)
    qhd_avg_runtime = response_qhd.info['timing']['qpu_access_time'] * 1e-6 / num_shots
    np.save(join(benchmark_dir, f"instance_{instance}/dwave_{device_label}_qhd_rez{resolution}_avg_runtime_{instance}.npy"), qhd_avg_runtime)
    # Save QAA samples
    qhd_filename = join(benchmark_dir, f"instance_{instance}/dwave_{device_label}_qhd_rez{resolution}_sample_{instance}.npy")
    with open(qhd_filename, 'wb') as f:
        np.save(f, qhd_samples)
    print(f"D-Wave standard QHD: data saved.")
    
    # Finish processing instance 
    print(f"Instance No. {instance} from {benchmark_name}: completed.\n")